# Processamento de Linguagem Natural

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.6


## Seq2seq

O Seq2seq foi introduzido pela primeira vez para tradução automática, pelo Google.
Antes disso, a tradução funcionava de maneira muito ingênua. Cada palavra que você
costumava digitar era convertida para o idioma de destino, sem considerar a gramática e a
estrutura da frase. O Seq2seq revolucionou o processo de tradução, utilizando o aprendizado
profundo (Deep Learning). Ele não apenas leva em consideração a palavra / entrada atual
durante a tradução, mas também sua vizinhança.
Atualmente, é usado para uma variedade de aplicações diferentes, como legendas de
imagens, modelos de conversação, resumo de texto, tradução, etc.

Como o nome sugere, seq2seq usa como entrada uma sequência de palavras (sentença
ou sentenças) e gera uma sequência de saída de palavras. Faz isso usando a rede neural
recorrente (RNN), sendo comum usarmos versões avançadas da RNN, ou seja, LSTM ou GRU
(estudadas no curso Deep Learning II). Isso ocorre porque a RNN sofre com o problema da
dissipação do gradiente. O modelo LSTM é usado na versão proposta pelo Google. Ele
desenvolve o contexto da palavra, recebendo 2 entradas em cada ponto do tempo. Um atual e
outro da saída anterior, daí o nome recorrente (a saída entra como entrada).
O Seq2seq possui principalmente dois componentes: codificador e decodificador, e,
portanto, às vezes é chamado de Rede Codificador-Decodificador.

* **Codificador:** Utiliza camadas de rede neural profunda e converte as palavras de entrada
em vetores ocultos correspondentes. Cada vetor representa a palavra atual e o contexto da
palavra.

* **Decodificador:** É semelhante ao codificador. Toma como entrada o vetor oculto gerado
pelo codificador, seus próprios estados ocultos e a palavra atual para produzir o próximo vetor
oculto e finalmente prever a próxima palavra.

Além desses dois elementos, muitas otimizações levaram a outros componentes do
seq2seq:

* **Attention:** A entrada para o decodificador é um único vetor que deve armazenar todas as
informações sobre o contexto. Isso se torna um problema com grandes sequências. Portanto, o
mecanismo de atenção é aplicado, permitindo que o decodificador observe a sequência de
entrada seletivamente.

* **Beam Search:** A palavra com maior probabilidade é selecionada como saída pelo decodificador.
Mas isso nem sempre produz os melhores resultados, devido ao problema básico dos
algoritmos gananciosos. Portanto, a pesquisa por feixe é aplicada, o que sugere possíveis
traduções em cada etapa. Isso é feito criando uma árvore dos melhores resultados.

* **Bucketing:** Sequências de comprimento variável são possíveis em um modelo seq2seq, devido
ao preenchimento de 0, que é feito na entrada e na saída. No entanto, se o comprimento
máximo definido por nós for 100 e a sentença tiver apenas 3 palavras, isso causará enorme
desperdício de espaço. Então, usamos o conceito de Bucketing. Criamos variáveis de tamanhos
diferentes, como (4, 8) (8, 15) e assim por diante, onde 4 é o comprimento máximo de entrada
definido por nós e 8 é o comprimento máximo de saída definido.

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install torch==1.5.0

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Instala o PyTorch
!pip install -q torch

In [ ]:
# O pacote torchtext fornece diversos datasets e funções para PLN
# https://torchtext.readthedocs.io/en/latest/index.html
!pip install -q torchtext

In [ ]:
# Instala o spacy
!pip install -q spacy

In [ ]:
# Imports
import math
import time
import spacy
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchtext
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

spacy     2.2.4
numpy     1.18.4
torch     1.5.0
torchtext 0.6.0
Data Science Academy


Nota: O treinamento do modelo deste estudo de caso é computacionalmente intensivo e por isso treinamos o modelo no Titan, o super servidor da DSA, com 3 GPUs e 128 GB de Memória RAM. O acesso a esse servidor é gratuito para alunos das Formações:

- <a href="https://www.datascienceacademy.com.br/pages/formacao-inteligencia-artificial">Formação Inteligência Artificial</a>
- <a href="https://www.datascienceacademy.com.br/pages/formacao-ia-aplicada-a-medicina">Formação Inteligência Artificial Aplicada à Medicina</a>
- <a href="https://www.datascienceacademy.com.br/pages/formacao-engenheiro-blockchain">Formação Engenheiro Blockchain</a>

O treinamento pode ser feito em um computador apenas com CPU. O tempo de treinamento será um pouco maior, mas o estudo de caso poderá ser executado sem problemas.

In [ ]:
# Aqui definimos o device que será usado para treinar o modelo
# Se pelo menos uma GPU estiver disponível, usaremos o device 'cuda' (nome da plataforma da Nvidia para GPU)
# Se não tiver GPU, usaremos a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Abaixo a descrição das GPUs do servidor da DSA. O comando abaixo funcionará somente se a plataforma CUDA da Nivida estiver instalada no computador. Se quiser conhecer mais sobre a plataforma CUDA, acesse aqui:

https://developer.nvidia.com/cuda-toolkit

In [ ]:
# GPUs no servidor da DSA
!nvidia-smi

Thu May 21 21:49:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    On   | 00000000:05:00.0 Off |                  N/A |
| 23%   41C    P8     9W / 250W |    125MiB / 12194MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  On   | 00000000:09:00.0 Off |                  N/A |
| 23%   38C    P8     9W / 250W |     12MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

### Carregando os Dicionários

Precisamos instalar os dicionários dos idiomas que serão usados para treinar o modelo. Aqui você encontra detalhes sobre os datasets:

https://www.statmt.org/wmt16/multimodal-task.html

In [ ]:
# Download do dicionário em inglês
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/dmpm/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/home/dmpm/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
# Download do dicionário em alemão
!python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/home/dmpm/anaconda3/lib/python3.7/site-packages/de_core_news_sm -->
/home/dmpm/anaconda3/lib/python3.7/site-packages/spacy/data/de
You can now load the model via spacy.load('de')


Agora carregamos os dicionários na memória.

In [ ]:
# Carregando os dicionários
spacy_german = spacy.load('de')
spacy_english = spacy.load('en')

Vamos criar duas funções para tokenização dos dicionários.

In [ ]:
# Função para tokenização do dicionário em inglês
def tokenize_english(text):
    return [token.text for token in spacy_english.tokenizer(text)][::-1]

In [ ]:
# Função para tokenização do dicionário em alemão
def tokenize_german(text):
    return [token.text for token in spacy_german.tokenizer(text)]

Precisamos agora criar a fonte e o destino, ou seja, o idioma fonte e o idioma destino para nosso tradutor.

Nosso modelo deverá fazer a tradução do inglês para o alemão. Inglês será a fonte (SOURCE) e Alemão será o destino (TARGET).

In [ ]:
# Idioma de origem
SOURCE = Field(tokenize = tokenize_english, init_token = '<sos>', eos_token = '<eos>', lower = True)

In [ ]:
# Idioma de destino
TARGET = Field(tokenize = tokenize_german, init_token = '<sos>', eos_token = '<eos>', lower = True)

In [ ]:
# Usamos a split() do pacote Multi30k do torchtext para separar os dicionários em SOURCE e TARGET
# e então em treino, validação e teste
# Obs: Será feito o download dos dados no pacote Multi30k
dados_treino, dados_valid, dados_teste = Multi30k.splits(exts = ('.en', '.de'), fields = (SOURCE, TARGET))

In [ ]:
# Visualizando os dados de treino, SOURCE e TARGET
print(dados_treino.examples[0].src)
print(dados_treino.examples[0].trg)

['.', 'bushes', 'many', 'near', 'outside', 'are', 'males', 'white', ',', 'young', 'two']
['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']


In [ ]:
print("Tamanho do Dataset de Treino: " + str(len(dados_treino.examples)))
print("Tamanho do Dataset de Validação: " + str(len(dados_valid.examples)))
print("Tamanho do Dataset de Teste: " + str(len(dados_teste.examples)))

Tamanho do Dataset de Treino: 29000
Tamanho do Dataset de Validação: 1014
Tamanho do Dataset de Teste: 1000


In [ ]:
# Vamos criar os vocabulários de SOURCE  e TARGET
SOURCE.build_vocab(dados_treino, min_freq = 2)
TARGET.build_vocab(dados_treino, min_freq = 2)

In [ ]:
# Print do tamanho dos vocabulários
print("Tamanho do Vocabulário em Inglês (SOURCE): " + str(len(SOURCE.vocab)))
print("Tamanho do Vocabulário em Alemão (TARGET): " + str(len(TARGET.vocab)))

Tamanho do Vocabulário em Inglês (SOURCE): 5893
Tamanho do Vocabulário em Alemão (TARGET): 7855


### Construindo o Modelo

Criaremos 3 classes:

- Encoder
- Decoder
- Seq2Seq

In [ ]:
# Classe para o Encoder
class Encoder(nn.Module):

    # Método construtor
    def __init__(self, input_dims, emb_dims, hid_dims, n_layers, dropout):
        super().__init__()

        # Camadas do modelo
        self.hid_dims = hid_dims
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dims, emb_dims)
        self.rnn = nn.LSTM(emb_dims, hid_dims, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)

    # Método forward para o treinamento
    def forward(self, src):

        # Execução do modelo
        embedded = self.dropout(self.embedding(src))
        outputs, (h, cell) = self.rnn(embedded)

        return h, cell

In [ ]:
# Classe para o Decoder
class Decoder(nn.Module):

    # Método construtor
    def __init__(self, output_dims, emb_dims, hid_dims, n_layers, dropout):
        super().__init__()

        # Camadas do modelo
        self.output_dims = output_dims
        self.hid_dims = hid_dims
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dims, emb_dims)
        self.rnn = nn.LSTM(emb_dims, hid_dims, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hid_dims, output_dims)
        self.dropout = nn.Dropout(dropout)

    # Método forward para o treinamento
    def forward(self, input, h, cell):

        # Execução do modelo
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (h, cell) = self.rnn(embedded, (h, cell))
        pred = self.fc_out(output.squeeze(0))

        return pred, h, cell

In [ ]:
# Classe para o modelo Seq2Seq
class Seq2Seq(nn.Module):

    # Método construtor
    def __init__(self, encoder, decoder, device):
        super().__init__()

        # Componentes do modelo
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    # Método forward para o treinamento
    def forward(self, src, trg, teacher_forcing_rate = 0.5):

        # Execução do modelo
        batch_size = trg.shape[1]
        target_length = trg.shape[0]
        target_vocab_size = self.decoder.output_dims
        outputs = torch.zeros(target_length, batch_size, target_vocab_size).to(self.device)
        h, cell = self.encoder(src)
        input = trg[0,:]

        for t in range(1, target_length):

            output, h, cell = self.decoder(input, h, cell)
            outputs[t] = output
            top = output.argmax(1)
            input = trg[t] if (random.random() < teacher_forcing_rate) else top

        return outputs

Vamos definir alguns hiperparâmetros e os gerados de dados.

In [ ]:
# Hiperparâmetros
batch_size = 32
input_dimensions = len(SOURCE.vocab)
output_dimensions = len(TARGET.vocab)
encoder_embedding_dimensions = 256
decoder_embedding_dimensions = 256
hidden_layer_dimensions = 512
num_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
epochs = 20
grad_clip = 1
lowest_validation_loss = float('inf')

In [ ]:
# Geradores de dados
iterador_treino, iterador_valid, iterador_teste = BucketIterator.splits((dados_treino, dados_valid, dados_teste),
                                                                        batch_size = batch_size,
                                                                        device = device)

Aqui nós criamos o encoder, decoder e o modelo:

In [ ]:
# Instância do Encoder
encod = Encoder(input_dimensions,
                encoder_embedding_dimensions,
                hidden_layer_dimensions,
                num_layers,
                encoder_dropout)

In [ ]:
# Instância do Decoder
decod = Decoder(output_dimensions,
                decoder_embedding_dimensions,
                hidden_layer_dimensions,
                num_layers,
                decoder_dropout)

In [ ]:
# Instância do Modelo
modelo = Seq2Seq(encod, decod, device).to(device)

In [ ]:
# Modelo criado
modelo

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=7855, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Vamos definir a função de inicalização dos pesos, função de custo e otimizador.

In [ ]:
# Precisamos de uma função para inicializar os pesos da rede neural
def inicializa_pesos(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.1, 0.1)

In [ ]:
# Incluímos a função de inicialização dos pesos no modelo
modelo.apply(inicializa_pesos)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=7855, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
# Definimos a função de custo para calcular o erro do modelo
criterion = nn.CrossEntropyLoss(ignore_index = TARGET.vocab.stoi[TARGET.pad_token])

In [ ]:
# Criamos o otimizador para atualizar os pesos do modelo a cada passada de treinamento
optimizer = optim.Adam(modelo.parameters())

Embora não seja obrigatório, criar funções para treino e avaliação do modelo ajuda a modularizar nosso processo de treinamento do modelo.

In [ ]:
# Função para treinar o modelo
def treina_modelo(modelo, iterator, optimizer, criterion, clip):

    # Inicia o método de treinamento
    modelo.train()

    # Inicializa o erro da epoch
    epoch_loss = 0

    # Loop pelo iterador (gerador de dados)
    for i, batch in enumerate(iterator):

        # Coletamos dados fonte e destino
        src = batch.src
        trg = batch.trg

        # Zeramos os gradientes
        optimizer.zero_grad()

        # Fazemos as previsões com o modelo
        output = modelo(src, trg)

        # Ajustamos o shape das previsões
        output_dims = output.shape[-1]
        output = output[1:].view(-1, output_dims)
        trg = trg[1:].view(-1)

        # Calculamos o erro do modelo
        loss = criterion(output, trg)

        # Iniciamos o backpropgation
        loss.backward()

        # Calculamos os gradientes da derivada para atualização dos pesos
        torch.nn.utils.clip_grad_norm_(modelo.parameters(), clip)

        # Aplicamos a atualização dos pesos
        optimizer.step()

        # Armazenamos o erro da epoch
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
# Função para avaliar o modelo
def avalia_modelo(modelo, iterator, criterion):

    # Inicia função de avaiação
    modelo.eval()

    # Inicializa o erro da epoch
    epoch_loss = 0

    # Vamos fazer as previsões com o modelo
    with torch.no_grad():

        # Loop pelo iterador (gerador de dados)
        for i, batch in enumerate(iterator):

            # Extrai fonte e destino
            src = batch.src
            trg = batch.trg

            # Previsão com o modelo
            output = modelo(src, trg, 0)

            # Ajusta as dimensões das previsões
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            # Calcula o erro do modelo
            loss = criterion(output, trg)

            # Armazena o erro na epoch
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

### Treinando o Modelo

O treinamento do modelo é demorado. Seja paciente.

In [ ]:
# Loop pelo número de epochs para treinar o modelo
for epoch in range(epochs):

    # Grava o tempo quando começamos
    start_time = time.time()

    # Treinamento
    train_loss = treina_modelo(modelo, iterador_treino, optimizer, criterion, grad_clip)

    # Validação
    valid_loss = avalia_modelo(modelo, iterador_valid, criterion)

    # Grava o tempo quando finalizamos
    end_time = time.time()

    # Verificamos o erro mínimo e então salvamos o modelo fazendo um checkpoint do modelo com melhor performance
    if valid_loss < lowest_validation_loss:
        lowest_validation_loss = valid_loss
        torch.save(modelo.state_dict(), 'modelos/seq2seq.pt')

    # Print
    print(f'Epoch: {epoch+1:02} | Time: {np.round(end_time-start_time,0)}s')
    print(f'\tErro em Treino: {train_loss:.4f}')
    print(f'\t Erro em Validação: {valid_loss:.4f}')

Epoch: 01 | Time: 48.0s
	Erro em Treino: 4.7076
	 Erro em Validação: 4.6512
Epoch: 02 | Time: 47.0s
	Erro em Treino: 3.9973
	 Erro em Validação: 4.3468
Epoch: 03 | Time: 47.0s
	Erro em Treino: 3.6368
	 Erro em Validação: 4.0514
Epoch: 04 | Time: 48.0s
	Erro em Treino: 3.3807
	 Erro em Validação: 3.8883
Epoch: 05 | Time: 48.0s
	Erro em Treino: 3.1684
	 Erro em Validação: 3.8140
Epoch: 06 | Time: 48.0s
	Erro em Treino: 2.9915
	 Erro em Validação: 3.6792
Epoch: 07 | Time: 47.0s
	Erro em Treino: 2.8233
	 Erro em Validação: 3.6726
Epoch: 08 | Time: 48.0s
	Erro em Treino: 2.6837
	 Erro em Validação: 3.6291
Epoch: 09 | Time: 48.0s
	Erro em Treino: 2.5523
	 Erro em Validação: 3.6190
Epoch: 10 | Time: 47.0s
	Erro em Treino: 2.4393
	 Erro em Validação: 3.5998
Epoch: 11 | Time: 47.0s
	Erro em Treino: 2.3307
	 Erro em Validação: 3.5683
Epoch: 12 | Time: 48.0s
	Erro em Treino: 2.2307
	 Erro em Validação: 3.5983
Epoch: 13 | Time: 48.0s
	Erro em Treino: 2.1253
	 Erro em Validação: 3.5880
Epoch: 14 | 

### Avaliando o Modelo

Com o modelo treinado, avaliamos com dados de teste.

In [ ]:
# Carregamos o modelo treinado
modelo.load_state_dict(torch.load('modelos/seq2seq.pt'))

<All keys matched successfully>

In [ ]:
# Avaliamos o modelo
test_loss = avalia_modelo(modelo, iterador_teste, criterion)

In [ ]:
# Print
print(f'Erro em Teste: {test_loss:.4f}')

Erro em Teste: 3.5024


### Traduzindo Idioma

Modelo treinado e avaliado, vamos usá-lo para o fim para o qual ele foi criado.

In [ ]:
# Função para tradução de idioma em 5 sentenças
def traduz_idioma(modelo, iterator, limit = 5):

    with torch.no_grad():

        # Loop pelo iterador
        for i, batch in enumerate(iterator):

            # Enquanto estivermos dentro do limite, vamos fazendo tradução
            if i < limit :

                # Extraímos SOURCE e TARGET
                # Fazemos isso para poder comparar a tradução correta com a previsão
                src = batch.src
                trg = batch.trg

                # Previsão do modelo
                output = modelo(src, trg, 0)

                # Todas as previsões
                preds = torch.tensor([[torch.argmax(x).item()] for x in output])

                # Prints
                print('Texto de Origem em Inglês: ' + str([SOURCE.vocab.itos[x] for x in src][1:-1][::-1]))
                print('Texto de Destino em Alemão (Valor Esperado): ' + str([TARGET.vocab.itos[x] for x in trg][1:-1]))
                print('Texto de Destino em Alemão (Valor Previsto): ' + str([TARGET.vocab.itos[x] for x in preds][1:-1]))
                print('\n')

In [ ]:
# Vamos gerar texto randômico a partir dos dados disponíveis
_, _, iterador_translate = BucketIterator.splits((dados_treino, dados_valid, dados_teste),
                                                 batch_size = 1,
                                                 device = device)

In [ ]:
# Tradução de idioma
saida = traduz_idioma(modelo, iterador_translate)

Texto de Origem em Inglês: ['two', 'men', 'wearing', 'hats', '.']
Texto de Destino em Alemão (Valor Esperado): ['zwei', 'männer', 'mit', 'mützen', '.']
Texto de Destino em Alemão (Valor Previsto): ['zwei', 'männer', 'mit', 'schwarzen', 'haaren']


Texto de Origem em Inglês: ['young', 'woman', 'climbing', 'rock', 'face']
Texto de Destino em Alemão (Valor Esperado): ['junge', 'frau', 'klettert', 'auf', 'felswand']
Texto de Destino em Alemão (Valor Previsto): ['eine', 'junge', 'frau', 'klettert', 'eine']


Texto de Origem em Inglês: ['a', 'woman', 'is', 'playing', 'volleyball', '.']
Texto de Destino em Alemão (Valor Esperado): ['eine', 'frau', 'spielt', 'volleyball', '.']
Texto de Destino em Alemão (Valor Previsto): ['eine', 'frau', 'spielt', 'volleyball', '.']


Texto de Origem em Inglês: ['three', 'men', 'are', 'walking', 'up', 'hill', '.']
Texto de Destino em Alemão (Valor Esperado): ['drei', 'männer', 'gehen', 'bergauf', '.']
Texto de Destino em Alemão (Valor Previsto): ['drei', 'männ

Parabéns! Aí está está seu tradutor de texto com Machine Learning e PLN.

# Fim